# Install catboost

In [ ]:
!pip install catboost

# Loading Libraries

In [ ]:
# Installation of required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, GridSearchCV

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

%config InlineBackend.figure_format = 'retina'
import numpy as np # Fundamental package for linear algebra and multidimensional arrays
import pandas as pd # Data analysis and manipulation tool

# to ignore warnings
import warnings
warnings.filterwarnings("ignore")

# to display all columns and rows:
pd.set_option('display.max_columns', None); pd.set_option('display.max_rows', None);

#  Loading and preparing training data

In [ ]:
# In read_csv() function, we have passed the location to where the files are located in the dphi official github page.
train_data = pd.read_csv("/content/drive/MyDrive/Dphi_Competition/Campus Recruitment/Train_Data.csv")

train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 670 entries, 0 to 669
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gender          670 non-null    object 
 1   ssc_p           670 non-null    float64
 2   ssc_b           670 non-null    object 
 3   hsc_p           670 non-null    float64
 4   hsc_b           670 non-null    object 
 5   hsc_s           670 non-null    object 
 6   degree_p        670 non-null    float64
 7   degree_t        670 non-null    object 
 8   etest_p         670 non-null    float64
 9   mba_p           670 non-null    float64
 10  specialisation  670 non-null    object 
 11  workex          670 non-null    object 
 12  status          670 non-null    object 
dtypes: float64(5), object(8)
memory usage: 68.2+ KB


In [ ]:
train_data.head()

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,etest_p,mba_p,specialisation,workex,status
0,M,73.976975,Central,77.392691,Others,Science,73.623151,Sci&Tech,84.498310,67.207006,Mkt&HR,Yes,Placed
1,M,62.000000,Others,62.000000,Others,Commerce,60.000000,Comm&Mgmt,63.000000,52.380000,Mkt&HR,Yes,Placed
2,M,80.965242,Others,67.104275,Others,Science,72.482621,Sci&Tech,91.343588,71.392313,Mkt&Fin,No,Placed
3,M,52.000000,Others,65.000000,Others,Arts,57.000000,Others,75.000000,59.810000,Mkt&Fin,Yes,Not Placed
4,F,69.000000,Central,62.000000,Central,Science,66.000000,Sci&Tech,75.000000,67.990000,Mkt&HR,No,Not Placed


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['status'] = le.fit_transform(train_data['status'])

In [ ]:
train_data.head()

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,etest_p,mba_p,specialisation,workex,status
0,M,73.976975,Central,77.392691,Others,Science,73.623151,Sci&Tech,84.498310,67.207006,Mkt&HR,Yes,1
1,M,62.000000,Others,62.000000,Others,Commerce,60.000000,Comm&Mgmt,63.000000,52.380000,Mkt&HR,Yes,1
2,M,80.965242,Others,67.104275,Others,Science,72.482621,Sci&Tech,91.343588,71.392313,Mkt&Fin,No,1
3,M,52.000000,Others,65.000000,Others,Arts,57.000000,Others,75.000000,59.810000,Mkt&Fin,Yes,0
4,F,69.000000,Central,62.000000,Central,Science,66.000000,Sci&Tech,75.000000,67.990000,Mkt&HR,No,0


# Data Pre-processing

In [ ]:
cat_df = train_data[["ssc_b", "hsc_b", "degree_t", "specialisation","workex","gender","hsc_s"]]
statu_df = train_data['status']

In [ ]:
# Input/independent variables
X = train_data.drop(["ssc_b", "hsc_b", "degree_t", "specialisation","workex","gender","hsc_s","status"], axis = 1)

y = train_data['status'] # Target variable

In [ ]:

cols= X.columns
index = X.index

In [ ]:
from sklearn.preprocessing import RobustScaler
transformer = RobustScaler().fit(X)
X = transformer.transform(X)
X = pd.DataFrame(X, columns =cols , index = index)

In [ ]:
X = pd.concat([X,cat_df], axis = 1)


In [ ]:
X = pd.concat([X,statu_df], axis = 1)


In [ ]:
X.head()

,ssc_p,hsc_p,degree_p,etest_p,mba_p,ssc_b,hsc_b,degree_t,specialisation,workex,gender,hsc_s,status
0,0.363904,1.153339,0.848631,0.598328,0.635794,Central,Others,Sci&Tech,Mkt&HR,Yes,M,Science,1
1,-0.408271,-0.251234,-0.477658,-0.489871,-1.359173,Others,Others,Comm&Mgmt,Mkt&HR,Yes,M,Commerce,1
2,0.814449,0.214528,0.737594,0.944822,1.198925,Others,Others,Sci&Tech,Mkt&Fin,No,M,Science,1
3,-1.052987,0.022514,-0.769725,0.117544,-0.359470,Others,Others,Others,Mkt&Fin,Yes,M,Arts,0
4,0.043031,-0.251234,0.106475,0.117544,0.741145,Central,Central,Sci&Tech,Mkt&HR,No,F,Science,0


In [ ]:
print(X.shape, y.shape)


(670, 13) (670,)


In [ ]:
train_data = pd.get_dummies(X)


In [ ]:
train_data.head()

,ssc_p,hsc_p,degree_p,etest_p,mba_p,status,ssc_b_Central,ssc_b_Others,hsc_b_Central,hsc_b_Others,degree_t_Comm&Mgmt,degree_t_Others,degree_t_Sci&Tech,specialisation_Mkt&Fin,specialisation_Mkt&HR,workex_No,workex_Yes,gender_F,gender_M,hsc_s_Arts,hsc_s_Commerce,hsc_s_Science
0,0.363904,1.153339,0.848631,0.598328,0.635794,1,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1
1,-0.408271,-0.251234,-0.477658,-0.489871,-1.359173,1,0,1,0,1,1,0,0,0,1,0,1,0,1,0,1,0
2,0.814449,0.214528,0.737594,0.944822,1.198925,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,1
3,-1.052987,0.022514,-0.769725,0.117544,-0.359470,0,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0
4,0.043031,-0.251234,0.106475,0.117544,0.741145,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,0,1


In [ ]:
train_data = train_data.drop('status',axis=1)

In [ ]:
train_data.head()

,ssc_p,hsc_p,degree_p,etest_p,mba_p,ssc_b_Central,ssc_b_Others,hsc_b_Central,hsc_b_Others,degree_t_Comm&Mgmt,degree_t_Others,degree_t_Sci&Tech,specialisation_Mkt&Fin,specialisation_Mkt&HR,workex_No,workex_Yes,gender_F,gender_M,hsc_s_Arts,hsc_s_Commerce,hsc_s_Science
0,0.363904,1.153339,0.848631,0.598328,0.635794,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1
1,-0.408271,-0.251234,-0.477658,-0.489871,-1.359173,0,1,0,1,1,0,0,0,1,0,1,0,1,0,1,0
2,0.814449,0.214528,0.737594,0.944822,1.198925,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,1
3,-1.052987,0.022514,-0.769725,0.117544,-0.359470,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0
4,0.043031,-0.251234,0.106475,0.117544,0.741145,1,0,1,0,0,0,1,0,1,1,0,1,0,0,0,1


In [ ]:
# Train-Test Separation
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data, y, 
                                                    test_size=0.3, 
                                                    random_state=12345)

In [ ]:
# Because it's an unstable data set, we're going to increase the number of samples.
# References: https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.combine.SMOTETomek.html
from imblearn.combine import SMOTETomek

smk = SMOTETomek()
# Oversample training  data
X_train, y_train = smk.fit_sample(X_train, y_train)

# Oversample validation data
X_test, y_test = smk.fit_sample(X_test, y_test)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(626, 21) (246, 21) (626,) (246,)


# Building Machine learning clasification Models

The most frequently used machine learning classification algorithms are : 


1.   Logistic Regression
2.   K nearest neighbors
3.   Decision Tree classifier
4.   Random Forest Classifier
5.   Support vector machine
6.   Gradient Boosting Classifier
7.   LGBM classifier 
8.   CatBoost Classifier



In [ ]:
models = []
models.append(('LR', LogisticRegression(random_state = 12345)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier(random_state = 12345)))
models.append(('RF', RandomForestClassifier(random_state = 12345)))
models.append(('SVM', SVC(gamma='auto', random_state = 12345)))
models.append(('XGB', GradientBoostingClassifier(random_state = 12345)))
models.append(("LightGBM", LGBMClassifier(random_state = 12345)))
models.append(("CatBoost", CatBoostClassifier(random_state = 12345, verbose = False)))

# evaluate each model in turn
results = []
names = []

In [ ]:
for name, model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        msg = "%s: (%f)" % (name, accuracy)
        print(msg)

LR: (0.951220)
KNN: (0.943089)
CART: (0.910569)
RF: (0.971545)
SVM: (0.967480)
XGB: (0.963415)
LightGBM: (0.959350)
CatBoost: (0.963415)


#LightGBM

In [ ]:
LightGBM = LGBMClassifier(random_state = 12345)
LightGBM = LogisticRegression(random_state = 12345)
LightGBM.fit(X_train, y_train)

y_pred = LightGBM.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9512195121951219


#Random Forest

In [ ]:
RF = RandomForestClassifier(random_state = 12345)

RF = LogisticRegression(random_state = 12345)
RF.fit(X_train, y_train)

y_pred = RF.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9428571428571428


In [ ]:
RF = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0 , max_depth=10)


RF = LogisticRegression(random_state = 12345)
RF.fit(X_train, y_train)

y_pred = RF.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9512195121951219


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [ ]:
from sklearn.ensemble import RandomForestRegressor


# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   49.7s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [ ]:
rf_random.best_params_
{'bootstrap': True,
 'max_depth': 70,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 400}

{'bootstrap': True,
 'max_depth': 70,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 400}

In [ ]:
RF = RandomForestRegressor(n_estimators = 400, random_state = 42,bootstrap=True,max_depth=70,max_features='auto',min_samples_leaf='4',min_samples_split='10')

RF = LogisticRegression(random_state = 12345)
RF.fit(X_train, y_train)

y_pred = RF.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9428571428571428


In [ ]:
RF = RandomForestRegressor(n_estimators = 10, random_state = 42,bootstrap=True,max_depth=10,max_features='auto',min_samples_leaf='4',min_samples_split='10')

RF = LogisticRegression(random_state = 12345)
RF.fit(X_train, y_train)

y_pred = RF.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9428571428571428


#LogisticRegression

In [ ]:
# Importing LogisticRegression from sklearn.linear_model
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(random_state = 12345)
LR.fit(X_train, y_train)

y_pred = LR.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9588235294117647


#CatBoost

In [ ]:
catboost =CatBoostClassifier(random_state = 12345, verbose = False, depth = 10, iterations = 1000, l2_leaf_reg = 5, learning_rate = 0.01)

catboost.fit(X_train, y_train)

y_pred = catboost.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)


0.9642857142857143


#LightGBM

In [ ]:
LightGBM =LGBMClassifier(random_state = 12345,  learning_rate = 0.05, max_depth = 3, n_estimators = 1000)

LightGBM.fit(X_train, y_train)

y_pred = LightGBM.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)


0.967479674796748


In [ ]:
models2 = []
models2.append(('CART', DecisionTreeClassifier( random_state = 12345)))
models2.append(('RF', RandomForestClassifier( random_state = 12345)))
models2.append(('XGB', GradientBoostingClassifier( random_state = 12345)))
models2.append(("LightGBM", LGBMClassifier( random_state = 12345)))
models2.append(("CatBoost", CatBoostClassifier(random_state = 12345, verbose = False)))

In [ ]:
for name, model in models2:
        base = model.fit(X_train,y_train)
        y_pred = base.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        msg = "%s: (%f)" % (name, accuracy)
        print(msg)

CART: (0.914286)
RF: (0.978571)
XGB: (0.964286)
LightGBM: (0.967857)
CatBoost: (0.964286)


In [ ]:
# Hyperparameters have previously been obtained with the help of GridSearchCV.
models = []
models.append(('XGB', GradientBoostingClassifier(random_state = 12345,learning_rate = 0.05, max_depth = 5, min_samples_split = 2, n_estimators = 500, subsample = 0.8)))
models.append(("LightGBM", LGBMClassifier(random_state = 12345,  learning_rate = 0.05, max_depth = 3, n_estimators = 1000)))
models.append(("CatBoost", CatBoostClassifier(random_state = 12345, verbose = False, depth = 10, iterations = 1000, l2_leaf_reg = 5, learning_rate = 0.01)))

# evaluate each model in turn
results = []
names = []

for name, model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        msg = "%s: (%f)" % (name, accuracy)
        print(msg)

XGB: (0.951220)
LightGBM: (0.967480)
CatBoost: (0.963415)


#XGB

In [ ]:
XGB = GradientBoostingClassifier(random_state = 12345,learning_rate = 0.05, max_depth = 5, min_samples_split = 2, n_estimators = 500, subsample = 0.8)

XGB.fit(X_train, y_train)

y_pred = XGB.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9705882352941176


# Testing on test data

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/Dphi_Competition/Campus Recruitment/Test_Data.csv')

In [ ]:
test_data.head()

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,etest_p,mba_p,degree_t,workex,specialisation
0,M,73.163372,Central,68.535901,Central,Science,71.581686,58.954215,58.235587,Comm&Mgmt,Yes,Mkt&Fin
1,M,74.000000,Central,70.000000,Central,Science,72.000000,60.000000,57.240000,Comm&Mgmt,Yes,Mkt&Fin
2,M,85.613691,Central,61.718334,Others,Science,70.962963,59.590873,60.635396,Sci&Tech,Yes,Mkt&HR
3,M,77.909979,Others,81.549151,Others,Science,69.847510,85.178574,64.304440,Sci&Tech,Yes,Mkt&Fin
4,M,80.851350,Others,66.168927,Others,Science,67.743252,87.851350,68.660478,Sci&Tech,Yes,Mkt&Fin


In [ ]:
# Input/independent variables
test_data_cat = test_data[["ssc_b", "hsc_b", "degree_t", "specialisation","workex","gender","hsc_s"]]

In [ ]:
test_data_cat.head()

,ssc_b,hsc_b,degree_t,specialisation,workex,gender,hsc_s
0,Central,Central,Comm&Mgmt,Mkt&Fin,Yes,M,Science
1,Central,Central,Comm&Mgmt,Mkt&Fin,Yes,M,Science
2,Central,Others,Sci&Tech,Mkt&HR,Yes,M,Science
3,Others,Others,Sci&Tech,Mkt&Fin,Yes,M,Science
4,Others,Others,Sci&Tech,Mkt&Fin,Yes,M,Science


In [ ]:
test_data = test_data.drop(["ssc_b", "hsc_b", "degree_t", "specialisation","workex","gender","hsc_s"], axis = 1)

In [ ]:
cols= test_data.columns
index = test_data.index

In [ ]:
from sklearn.preprocessing import RobustScaler
transformer = RobustScaler().fit(test_data)
test_data = transformer.transform(test_data)
test_data = pd.DataFrame(test_data, columns =cols , index = index)

In [ ]:
test_data = pd.concat([test_data,test_data_cat], axis = 1)


In [ ]:
test_data = pd.get_dummies(test_data)

In [ ]:
################################"
target = RF.predict(test_data)
################################

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
# In read_csv() function, we have passed the location to where the files are located in the dphi official github page.
train_data = pd.read_csv("/content/drive/MyDrive/Dphi_Competition/Campus Recruitment/Train_Data.csv")
y = train_data['status']
le.fit(y)

target = le.inverse_transform(target)

#Save prediction results locally via colab notebook

In [ ]:
# To create Dataframe of predicted value with particular respective index
res = pd.DataFrame(target) # target are nothing but the final predictions of your model on input features of your new unseen test data
res.columns = ["prediction"]

# To download the csv file locally
from google.colab import files
res.to_csv('submission.csv', index = False)
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#For people who work locally : Save prediciton results locally via jupyter notebook

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions}) # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False) # the csv file will be saved locally on the same location where this notebook is located.